In [115]:
import ipywidgets
import pandas as pd
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from ipywidgets import interact, Dropdown, IntSlider,Label
from IPython.display import display, clear_output

In [39]:
df = pd.read_csv('../datasets/tidy_data.csv')

In [160]:
def get_games_info(season_df,type_game):
    df = season_df[season_df['type_season']==type_game]
    ids = pd.unique(df['id_game']).tolist()
    d = dict()
    for id in ids:
         
        d[id]=dict()
        game_df = df[df['id_game']==id]
        d[id]['home'] = game_df.iloc[0]['team_home_name']
        d[id]['away'] = game_df.iloc[0]['team_away_name']
        event_ids = pd.unique(game_df['event_Idx']).tolist()
        for event_id in event_ids:
            event = game_df[df['event_Idx']==event_id]
            event_dict =dict()
            event_dict['shot_type'] = event['shot_type'].values[0]
            event_dict['is_goal'] = event['result_event'].values[0] == 'Goal'
            event_dict['player'] = event['shooter_name'].values[0]
            event_dict['x'] = event['x_coord'].values[0]
            event_dict['y'] = event['y_coord'].values[0]
            d[id][event_id] = event_dict     
    return d

In [161]:
teams = pd.unique(df['team_away_name']).tolist()
seasons = pd.unique(df['season']).tolist()
seasons = [20162017]
d = dict()
for season in seasons:
    season_df = df[df['season']==season]
    d[season] = dict()
    d[season]['playoffs']=get_games_info(season_df,'playoffs')
    d[season]['regular season']=get_games_info(season_df,'regular_season') 
    
            


C:\Users\xavie\AppData\Local\Temp\ipykernel_30448\3609880968.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  event = game_df[df['event_Idx']==event_id]


In [162]:
season =  20162017
seasonW = Dropdown(options = seasons, value = season)

game_typeW = Dropdown(options=['playoffs','regular season'], value = 'regular season')
game_type = 'regular season' 
games = sorted(d[season][game_type].keys())
game = games[0]
events_keys = list(d[season][game_type][game].keys())
events_keys.remove('home')
events_keys.remove('away')
events =sorted(events_keys)
event = events[0]

gameW = Dropdown(options = games,value=game)
eventW = Dropdown(options = events,value=event)


@interact(season = seasonW,game_type = game_typeW,game = gameW,event=eventW)
def plot(season,game_type,game,event):
    
    games = sorted(d[season][game_type].keys())
    gameW.options = games
    if not game in games:
        game = games[0]
        gameW.value = game
   
    events_keys = list(d[season][game_type][game].keys())
    events_keys.remove('home')
    events_keys.remove('away')
    events = sorted(events_keys)
    eventW.options = events
    if not event in events:
        event = events[0]
        eventW.value = event
    
   
    fig, ax = plt.subplots()
    img = plt.imread("../figures/nhl_rink.png")
    ax.imshow(img, extent=[-100, 100, -50, 50])
    
    season_title = f'{str(season)[:4]}-{str(season)[4:]}'
    home = d[season][game_type][game]['home']
    away = d[season][game_type][game]['away']
    plt.suptitle(f'{season_title}\n\n{home} VS {away}')
    
    shot_type = d[season][game_type][game][event]['shot_type']
    isGoal = d[season][game_type][game][event]['is_goal']
    player = d[season][game_type][game][event]['player']
    
   
    if isGoal:  
        plt.title(f'A {shot_type} of {player}: GOAL!')
    else:
        plt.title(f'A {shot_type} of {player}: Missed')   
   
    
    x = d[season][game_type][game][event]['x']
    y = d[season][game_type][game][event]['y']
    ax.scatter(x, y, marker='o', color='r', label='point')
    
    clear_output(wait=True)

interactive(children=(Dropdown(description='season', options=(20162017,), value=20162017), Dropdown(descriptio…